In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define your function
def MyFunction(fileinput):
    print(f"Processing file: {fileinput}")
    
    # Open the dataset with decode_times=False to avoid time parsing errors
    ds = xr.open_dataset(fileinput, decode_times=False)
    
    # Coordinates of the center of the burned area
    burned_area_lat = 41.709377863541654
    burned_area_lon = 1.892273844304144

    # Check if the dataset contains the necessary variables
    if 'latitude' not in ds or 'longitude' not in ds:
        print(f"File {fileinput} is missing 'latitude' or 'longitude'. Skipping.")
        return None
    
    # Calculate the nearest grid point indices
    abs_diff_lat = np.abs(ds.latitude.values - burned_area_lat)
    abs_diff_lon = np.abs(ds.longitude.values - burned_area_lon)

    # Find the index of the nearest grid point
    lat_idx = abs_diff_lat.argmin()
    lon_idx = abs_diff_lon.argmin()

    # Check for valid_time or similar time dimension
    if 'valid_time' in ds:
        july_data = ds.sel(valid_time=ds.valid_time.dt.month == 7)
        print(f"July data extracted for file: {fileinput}")
    else:
        print(f"No 'valid_time' dimension in {fileinput}. Using entire dataset.")
        july_data = ds

    # Extract the required variables if they exist
    if {'si10', 't2m', 'wdir10', 'r2'}.issubset(ds.data_vars):
        july_wind_speed_at_point = july_data.isel(latitude=lat_idx, longitude=lon_idx)[['si10', 't2m', 'wdir10', 'r2']]
        df = july_wind_speed_at_point.to_dataframe()
        return df
    else:
        print(f"Required variables missing in file: {fileinput}")
        return None

# List to store the dataframes from each file
df_all = []

# Assuming 'myallfiles' is a list of file paths
myallfiles = ['/data/IMFSE/PythonCourse/MNH/FCAST_model1.nc', 
              '/data/IMFSE/PythonCourse/MNH/FCAST_model2.nc', 
              '/data/IMFSE/PythonCourse/MNH/FCAST_model3.nc']  
              
# Loop through each file, process, and append the result
for file in myallfiles:
    df = MyFunction(file)
    if df is not None:
        df_all.append(df)
    else:
        print(f"Skipped file due to missing data: {file}")

# Check if any dataframes were successfully collected
if df_all:
    # Concatenate all the dataframes
    dfba = pd.concat(df_all)
    
    # Optional: View the concatenated dataframe
    print(dfba)

    # Plot histograms for wind speed ('si10'), temperature ('t2m'), wind direction ('wdir10'), and humidity ('r2')
    plt.figure(figsize=(12, 12))

    # Histogram for wind speed (si10)
    plt.subplot(2, 2, 1)
    plt.hist(dfba['si10'].dropna(), bins=30, color='b', alpha=0.7)
    plt.title('Histogram of Wind Speed (si10)')
    plt.xlabel('Wind Speed (si10)')
    plt.ylabel('Frequency')

    # Histogram for temperature (t2m)
    plt.subplot(2, 2, 2)
    plt.hist(dfba['t2m'].dropna(), bins=30, color='r', alpha=0.7)
    plt.title('Histogram of Temperature (t2m)')
    plt.xlabel('Temperature (t2m)')
    plt.ylabel('Frequency')

    # Histogram for wind direction (wdir10)
    plt.subplot(2, 2, 3)
    plt.hist(dfba['wdir10'].dropna(), bins=30, color='g', alpha=0.7)
    plt.title('Histogram of Wind Direction (wdir10)')
    plt.xlabel('Wind Direction (wdir10)')
    plt.ylabel('Frequency')

    # Histogram for humidity (r2)
    plt.subplot(2, 2, 4)
    plt.hist(dfba['r2'].dropna(), bins=30, color='c', alpha=0.7)
    plt.title('Histogram of Humidity (r2)')
    plt.xlabel('Humidity (r2)')
    plt.ylabel('Frequency')

    # Show the plots
    plt.tight_layout()
    plt.show()
else:
    print("No data to concatenate and plot.")


Processing file: /data/IMFSE/PythonCourse/MNH/FCAST_model1.nc
File /data/IMFSE/PythonCourse/MNH/FCAST_model1.nc is missing 'latitude' or 'longitude'. Skipping.
Skipped file due to missing data: /data/IMFSE/PythonCourse/MNH/FCAST_model1.nc
Processing file: /data/IMFSE/PythonCourse/MNH/FCAST_model2.nc
File /data/IMFSE/PythonCourse/MNH/FCAST_model2.nc is missing 'latitude' or 'longitude'. Skipping.
Skipped file due to missing data: /data/IMFSE/PythonCourse/MNH/FCAST_model2.nc
Processing file: /data/IMFSE/PythonCourse/MNH/FCAST_model3.nc
File /data/IMFSE/PythonCourse/MNH/FCAST_model3.nc is missing 'latitude' or 'longitude'. Skipping.
Skipped file due to missing data: /data/IMFSE/PythonCourse/MNH/FCAST_model3.nc
No data to concatenate and plot.
